In [38]:
# 并查集
# bug 统计
# 1: 并查集写错了
# 2: 考虑情况不周全,1在其他地方
# 3: 数量计算错误
# 4: 未考虑首位为零的情况
from heapq import heapify, heappush, heappop

class DisjointSet(object):
    def __init__(self, m, n):
        self.parent = [-1 for _ in range(m*n)]
        return

    def findParent(self, i):
        root = i
        while root != self.parent[root]:
            root = self.parent[root]
        
        while i!= root:
            tmp = self.parent[i]
            self.parent[i] = root
            i = tmp
        return root

    def union(self, i, j):
        iroot = self.findParent(i)
        jroot = self.findParent(j)
        self.parent[jroot] = iroot
        return

    def add(self, i):
        if self.parent[i] == -1:
            self.parent[i] = i 
        return

    def judge_path(self):
        parent = -1
        # print("what")
        for i in self.parent:
            if i != -1:
                np = self.findParent(i)
                if parent != -1 and parent != np:
                    return False
                parent = np
        return True

class Graph(object):
    def __init__(self):
        self.vertex = {}
        self.value = []
        heapify(self.value)
        self.count = 0
        self.pathcount = 0
        return

    def addvalue(self, i):
        if self.vertex.get(i) is None:
            heappush(self.value, i)
            self.vertex[i] = set()
            self.count += 1 if i != 1 else 0
        return

    def addvertex(self, i, j):
        self.addvalue(i)
        self.addvalue(j)
        self.vertex[i].add(j)
        self.vertex[j].add(i)
        return

    def findpath(self, i, j):  # 有个优化方法时保存路径值
        # BFS算法
        if j in self.vertex[i]:
            return 1
        visited = set()
        visited.add(i)
        stack = set()
        count = 1
        stack = self.vertex[i]
        while stack:
            tmp = set()
            count += 1
            for x in stack:
                if x not in visited:
                    for y in self.vertex[x]:
                        if y not in visited:
                            tmp.add(y)
                        if y == j:
                            return count
                visited.add(x)
            stack = tmp
        return -1

    def count_all(self, start):
        p = heappop(self.value)
        if p == 1:
            p = heappop(self.value)
        self.pathcount += self.findpath(start, p) if start!=p else 0
        while len(self.value) > 0:
            q = heappop(self.value)
            self.pathcount += self.findpath(p, q)
            p = q
        return self.pathcount
        
    
di, dj = [1, 0], [0, 1]
class Solution(object):
    def cutOffTree(self, forest):
        """
        :type forest: List[List[int]]
        :rtype: int
        """
        if forest[0][0] == 0: return -1
        self.m, self.n = len(forest), len(forest[0])
        self.ds = DisjointSet(self.m, self.n)
        self.g = Graph()
        for i in range(self.m):
            for j in range(self.n):
                if forest[i][j] != 0:
                    # self.g.addvalue(forest[i][j])
                    for k in range(2):
                        x, y = i+di[k], j+dj[k]
                        self.ds.add(i+j*self.m)
                        if 0<=x<self.m and 0<=y<self.n and forest[x][y]!=0:
                            self.g.addvertex(forest[i][j], forest[x][y])
                            self.ds.add(x+y*self.m)
                            self.ds.union(i+j*self.m, x+y*self.m)
        if self.ds.judge_path():
            return self.g.count_all(forest[0][0])
        return -1
        
    

In [40]:
# 纯粹的bfs
import collections
class Solution:
    def cutOffTree(self, forest) -> int:
        M = len(forest)
        N = len(forest[0])
        trees = []
        for i in range(M):
            for j in range(N):
                if (forest[i][j] > 1):
                    trees.append((forest[i][j], i, j))
        trees.sort()
        preX, preY = 0, 0
        res = 0
        for height, curX, curY in trees:
            steps = self.bfs(forest, preX, preY, curX, curY)
            if steps == -1:
                return -1
            res += steps
            preX, preY = curX, curY
        return res
    
    def bfs(self, forest, startX, startY, targetX, targetY):
        M = len(forest)
        N = len(forest[0])
        dirs = [(0, 1), (0, -1), (-1, 0), (1, 0)]
        steps = 0
        queue = collections.deque()
        queue.append((startX, startY))
        visited = set()
        visited.add((startX, startY))
        while queue:
            size = len(queue)
            for _ in range(size):
                curX, curY = queue.popleft()
                if curX == targetX and curY == targetY:
                    return steps
                for d in dirs:
                    nX = curX + d[0]
                    nY = curY + d[1]
                    if 0 <= nX < M and 0 <= nY < N and forest[nX][nY] != 0 and ((nX, nY) not in visited):
                        queue.append((nX, nY))
                        visited.add((nX, nY))
            steps += 1
        return -1

# 作者：fuxuemingzhu
# 链接：https://leetcode.cn/problems/cut-off-trees-for-golf-event/solution/by-fuxuemingzhu-dtet/
# 来源：力扣（LeetCode）
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

In [41]:
import unittest


class LeetcodeTest(unittest.TestCase):
    solution = getattr(Solution(), dir(Solution)[-1])

    def test_case1(self):
        test_input = [[1,2,3],[0,0,4],[7,6,5]]
        test_output = 6
        self.assertEqual(self.solution(test_input), test_output)
    def test_case2(self):
        test_input = [[1,2,3],[0,0,0],[7,6,5]]
        test_output = -1
        self.assertEqual(self.solution(test_input), test_output)
    def test_case3(self):
        test_input = [[54581641,64080174,24346381,69107959],[86374198,61363882,68783324,79706116],[668150,92178815,89819108,94701471],[83920491,22724204,46281641,47531096],[89078499,18904913,25462145,60813308]]
        test_output = 57
        self.assertEqual(self.solution(test_input), test_output)

unittest.main(argv=['ignored', '-v'], exit=False)

test_case1 (__main__.LeetcodeTest) ... ok
test_case2 (__main__.LeetcodeTest) ... ok
test_case3 (__main__.LeetcodeTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
